In [ ]:
import math
import os

import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from config import Config
from datagen import DataGenerator, _relative_points, _pad_img
from sphinx import SphinxModel
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
cfg = Config()

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
cfg.load = "sphinx_9"

dataset = DataGenerator(cfg)
dataset.generate_set(train = True)

In [ ]:
model = SphinxModel(cfg, dataset)
model.generate_model(True)

In [ ]:
with tf.name_scope('Session'):
    model.Session = tf.Session()
    model._init_variable()

In [ ]:
def _relative_points(points, shape):
    new_p = np.copy(points)
    w, h, _ = shape
    for i in range(len(new_p)):
        if (new_p[i] == [-1, -1]).any():
            continue
        if h > w:
            offset = math.floor((h - w) / 2)
            new_p[i, 0] += offset
        else:
            offset = math.floor((w - h) / 2)
            new_p[i, 1] += offset
    return new_p

In [ ]:
i = 0

In [ ]:
# get image info
name = dataset.valid_set[i]
point = dataset.data_dict[name]['points']
label = dataset.data_dict[name]['label']
weight = np.asarray(dataset.data_dict[name]['weight'])

# opencv width height is inversed
swapped_p = np.zeros(point.shape)
swapped_p[:, 0] = point[:, 1]
swapped_p[:, 1] = point[:, 0]

# open image
img = dataset.open_img(dataset.train_img_dir, name)
orig_size = max(img.shape)

# pad image and points
img = _pad_img(img)
new_p = _relative_points(swapped_p, img.shape)

# visualize ground truth
gt_hm_visual = dataset._generate_hm(orig_size, cfg.hm_size, new_p, weight, keep_invisible=False)
gt_hm_visual = np.sum(gt_hm_visual, 2) * 255
gt_hm_visual = cv2.resize(gt_hm_visual, (orig_size, orig_size), interpolation=cv2.INTER_LINEAR)
gt_hm_visual = np.expand_dims(gt_hm_visual, 2)
gt_visual = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) + gt_hm_visual.astype(np.int32)
pred_visual = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# resize image for inference
img = cv2.resize(img, (cfg.img_size, cfg.img_size), interpolation=cv2.INTER_LINEAR)
img = img.astype(np.float32) / 255
images = np.expand_dims(img, 0)

# get prediction
prediction = model.Session.run(model.output, feed_dict={model.img: images})
pred = prediction[0, model.nStacks - 1, :, :, :]

# calculate predictions from heatmap
pred_idxes = np.zeros((model.num_points, 2), dtype=np.float32)
total_dist = 0.0
for j in range(model.num_points):
    if weight[j] == 1:
        index = np.unravel_index(pred[:, :, j].argmax(), (model.hm_size, model.hm_size))
        pred_idx = np.array(index) / model.hm_size * orig_size
        
        pred_idxes[j, :] = pred_idx
        total_dist += np.linalg.norm(pred_idx - new_p[j])

# opencv width height is inversed
swapped_p = np.zeros(pred_idxes.shape)
swapped_p[:, 0] = pred_idxes[:, 1]
swapped_p[:, 1] = pred_idxes[:, 0]

# visualize prediction
pred_hm_visual = dataset._generate_hm(orig_size, cfg.hm_size, swapped_p, weight, keep_invisible=False)
pred_hm_visual = np.sum(pred_hm_visual, 2) * 255
pred_hm_visual = cv2.resize(pred_hm_visual, (orig_size, orig_size), interpolation=cv2.INTER_LINEAR)
pred_hm_visual = np.expand_dims(pred_hm_visual, 2)
pred_visual = pred_visual + pred_hm_visual.astype(np.int32)        
  
# calc normalized average error
if label <= 2:
    norm_idx1 = new_p[5]
    norm_idx2 = new_p[6]
else:
    norm_idx1 = new_p[15]
    norm_idx2 = new_p[16]

norm_dist = np.linalg.norm(norm_idx2 - norm_idx1)
error = total_dist / norm_dist / np.sum(np.count_nonzero(weight))

# show results
plt.imshow(gt_visual)
plt.show()
plt.imshow(pred_visual)
plt.show()
print("image error:", error)
        
i += 1